# IMDB Sentiment Classification Task using BERT Embeddings

In [ ]:
import keras
import tensorflow
import pickle
from keras.datasets import imdb, reuters
from keras.utils import to_categorical
from keras import models, optimizers, preprocessing
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, Embedding, SimpleRNN
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras import models, optimizers, Input
import numpy as np

from bert_serving.client import BertClient

from keras import models, optimizers, Input
from keras.models import Model
from keras.layers import Dense, Activation, Dropout, Flatten, Embedding, concatenate, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras import regularizers

from sklearn.metrics import f1_score, precision_score, recall_score
from keras.utils import np_utils


In [2]:
# (train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=100)
import os 
imdb_dir = './aclImdb/' 
train_dir = os.path.join(imdb_dir, 'train') 
labels = []
texts = [] 
count_pos = 0
count_neg = 0

for label_type in ['neg', 'pos']: 
    dir_name = os.path.join(train_dir, label_type) 
    for fname in os.listdir(dir_name): 
        if fname[-4:] == '.txt': 
            f = open(os.path.join(dir_name, fname)) 
            texts.append(f.read())
            f.close() 
            if label_type == 'neg': 
                labels.append(0)
            else: 
                labels.append(1) 

# Run Bert server by following guidelines here

https://github.com/hanxiao/bert-as-service

or to obtain elmo-like embeddings you can server using : 
```
bert-serving-start -pooling_strategy NONE -max_seq_len 256 -model_dir multi_cased_L-12_H-768_A-12

```
multi_cased_L-12_H-768_A-12 : this is model download from bert repository. you can download other models and try those also.


you have to install bert-server and bert client using :

```
pip install bert-serving-server  # server
pip install bert-serving-client  # client, independent of `bert-serving-server`

```

In [3]:
# train_data
bc = BertClient(check_length=False)
claims_vec = bc.encode(texts)

In [4]:
labels = np.asarray(labels)

In [6]:
class Metrics(Callback):

	def on_train_begin(self, logs={}):

		self.val_f1s = []
		self.val_recalls = []
		self.val_precisions = []
	 
	def on_epoch_end(self, epoch, logs={}):

		val_predict = (np.asarray(model.predict([claims_vec]))).round()
		val_targ = labels
		_val_f1 = f1_score(val_targ, val_predict, average = 'binary')
		_val_recall = recall_score(val_targ, val_predict, average ='binary')
		_val_precision = precision_score(val_targ, val_predict,average = 'binary')
		self.val_f1s.append(_val_f1)
		self.val_recalls.append(_val_recall)
		self.val_precisions.append(_val_precision)
		print (' — val_f1: %f — val_precision: %f — val_recall %f' %( _val_f1, _val_precision, _val_recall))
		return



class TrainModel:

    def lstm_model(self, claim_length, embedding_dim, nb_classes):


            claims_input = Input(shape=(claim_length, embedding_dim), dtype='float32', name='claims')
            encoded_claims = LSTM(32, return_sequences=True, recurrent_dropout=0.4)(claims_input)
            encoded_claims = LSTM(16, recurrent_dropout=0.4)(claims_input)

            concatenate_layers = Dense(8, kernel_regularizer=regularizers.l2(0.001), activation='relu')(encoded_claims)
            # concatenate_layers = Dropout(0.6)(concatenate_layers)
            # concatenate_layers = Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu')(concatenate_layers)
            # concatenate_layers = Dense(32, kernel_regularizer=regularizers.l2(0.001), activation='relu')(concatenate_layers)

            # concatenate_layers = Dropout(0.6)(concatenate_layers)
            if nb_classes == 3:

                pred_label = Dense(3, activation='softmax')(concatenate_layers)

            else:

                pred_label = Dense(1, activation='sigmoid')(concatenate_layers)



            return claims_input, pred_label
    
    
training = TrainModel()
metrics = Metrics()

max_claims_length = 138
max_sents_length = 138

nb_classes = 1

if nb_classes == 3:
    labels = np_utils.to_categorical(labels, nb_classes)
    loss = 'categorical_crossentropy'

else:
    loss = 'binary_crossentropy'

embedding_dim = 768
claims_input, pred_label = training.lstm_model(max_claims_length, embedding_dim, nb_classes)


model = Model([claims_input], pred_label)
print (model.summary())

early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.compile(optimizer=optimizers.Adam(), loss=loss, metrics=['accuracy'])

history = model.fit({'claims': claims_vec}, labels, 
                            epochs=5, batch_size=32, validation_split=0.1, callbacks=[early_stopping, metrics,
                                            ModelCheckpoint(filepath='model_imdb_lstm_bert.h5', monitor='val_loss', save_best_only=True)])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
claims (InputLayer)          (None, 138, 768)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 16)                50240     
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 50,385
Trainable params: 50,385
Non-trainable params: 0
_________________________________________________________________
None
Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 379s 17ms/step - loss: 0.4838 - acc: 0.7680 - val_loss: 0.6870 - val_acc: 0.6892
 — val_f1: 0.804923 — val_precision: 0.926555 — val_recall 0.711520
Epoch 2/5
